In [1]:
!pip install bing-image-downloader


You should consider upgrading via the 'c:\users\win10\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


In [2]:
!mkdir images


In [3]:
from bing_image_downloader import downloader
downloader.download("cat", limit=30, output_dir="images", adult_filter_off=True)
downloader.download("dog", limit=30, output_dir="images", adult_filter_off=True)


[%] Downloading Images to d:\image recognition\cat_dog_Recognition\images\cat


[!!]Indexing page: 1

[%] Indexed 30 Images on Page 1.


[%] Downloading Image #1 from https://www.fetchfind.com/blog/wp-content/uploads/2017/08/cat-2734999_1920-5-common-cat-sounds.jpg
[%] File Downloaded !

[%] Downloading Image #2 from https://wallsdesk.com/wp-content/uploads/2018/03/Cat-High-Definition-Wallpapers-2.jpg
[%] File Downloaded !

[%] Downloading Image #3 from http://images4.fanpop.com/image/photos/16100000/Cute-Kitten-kittens-16122951-1280-800.jpg
[%] File Downloaded !

[%] Downloading Image #4 from http://www.jonas-kyratzes.net/wp-content/uploads/2012/09/Cat2.jpg
[%] File Downloaded !

[%] Downloading Image #5 from https://i.pinimg.com/originals/e2/19/ae/e219aefe0b3eb25ca028d9616f21d437.jpg
[%] File Downloaded !

[%] Downloading Image #6 from http://2.bp.blogspot.com/_8UxqOB8rDCg/S7I4g2TuKgI/AAAAAAAABWM/-HyZ7oCwJ-0/s1600/Cat.JPG
[%] File Downloaded !

[%] Downloading Image #7 from https://c

In [14]:
# import os
# import numpy as np
# import matplotlib.pyplot as plt
# from skimage.io import imread
# from skimage.transform import resize

# # flatten the image from 2d to 1d 

# target=[]     #will describe whether dog or cat
# flat_data=[]  #flattening the data
# images=[]     #image to be processed

# types=["dog","cat"]

# #iterate the images from the array

# for pet in types:
   
#     class_num=types.index(pet)
#     path=os.path.join('./images',pet)

#     # list out the particular thing from the path
#     for img in os.listdir(path):
#         img_array=imread(os.path.join(path,img))  #will give the path of the images which need to be traversed
        
#         img_resized=  resize(img_array,(150,150,3))
#         flat_data.append(img_resized.flatten())
#         images.append(img_resized)
#         target.append(class_num)


#     flat_data=np.array(flat_data)
#     target=np.array(target)
#     images=np.array(image


# to build the model(preprocessing)

import os
import matplotlib.pyplot as plt
import numpy as np
from skimage.io import imread          #to read the images
from skimage.transform import resize   #to reframe the image in specific size

#to flatten the image from height and width to 1 dimension

target =[] #it will tell wether it is a icecream ball or sunflower
images=[]  #data to be processed
flat_data=[] #flattening the data

CATAGORIES=['dog','cat']

# itereate all the images

for category in CATAGORIES:
  print(CATAGORIES.index(category))  #to get the index of the element in the arrat
  class_num=CATAGORIES.index(category)
  path=os.path.join('./images',category) #create path to use all the image
  print(path)
  for img in os.listdir(path):  #to listout a particular thing from the path
    img_array=imread(os.path.join(path,img))
    # print(img_array.shape)  #will print the dimension like height width and depth of the images
    # plt.imshow(img_array)

#preprocessing:
# 1.resize
# 2.flattening

# 1.resize
    img_resized=  resize(img_array,(150,150,3))
    flat_data.append(img_resized.flatten())
    images.append(img_resized)
    target.append(class_num)

# if our values does not converted into numpy arrays

flat_data=np.array(flat_data)
target=np.array(target)
images=np.array(images)

0
./images\dog
1
./images\cat


In [15]:
# splitting the data into training and testing

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn import svm #its an algorithm
x_train, x_test, y_train, y_test=train_test_split(flat_data, target, test_size=0.3, random_state=108)

param_grid=[
    {'C':[1,10,1000], 'kernel':['linear']},
    {'C':[1,10,1000], 'gamma': [0.001,0.0001] ,'kernel':['rbf']}   
]

svc=svm.SVC(probability=True)
clf=GridSearchCV(svc, param_grid)
clf.fit(x_train, y_train)




GridSearchCV(estimator=SVC(probability=True),
             param_grid=[{'C': [1, 10, 1000], 'kernel': ['linear']},
                         {'C': [1, 10, 1000], 'gamma': [0.001, 0.0001],
                          'kernel': ['rbf']}])

In [23]:
y_pred=clf.predict(x_test)
from sklearn.metrics import accuracy_score,confusion_matrix
accuracy_score(y_pred,y_test)
confusion_matrix(y_pred,y_test)

array([[ 6, 12],
       [ 0,  0]], dtype=int64)

In [26]:
# deploy the Model

import pickle
pickle.dump(clf, open('image.p','wb'))
model=pickle.load(open('image.p','rb'))  #to check whether the dumb file is tored or not

In [1]:
%%writefile app.py
import numpy as np
from PIL import Image
import streamlit as st
from skimage.io import imread
from skimage.transform import resize
import pickle
st.title('Image classifier using machine learning')
st.text('upload the image')
model=pickle.load(open('image.p','rb'))

uploaded_file=st.file_uploader("choose an image...",type="jpg")
if uploaded_file is not None:
    img=Image.open(uploaded_file)
    st.image(img,caption="Uploaded Image")
    if st.button('PREDICT'):
        CATAGORIES=['dog','cat']
        st.write('Result..')
        flat_data=[]
        img=np.array(img)
        img_resized=resize(img,(150,150,3))
        flat_data.append(img_resized.flatten())
        flat_data=np.array(flat_data)
        y_out=model.predict(flat_data)
        y_out=CATAGORIES[y_out[0]]
        st.write(' PREDICTED OUTPUT -:', y_out)
        q=model.predict_proba(flat_data)
        for index ,item in enumerate(CATAGORIES):
            st.write(f'{item}:{q[0][index]*100}')


Overwriting app.py


In [3]:
from pyngrok import ngrok
# !nohub streamlit run app.py &
url=ngrok.connect(port='8506')
url

t=2023-01-28T22:03:35+0530 lvl=warn msg="failed to start tunnel" pg=/api/tunnels id=6e1434b3d7f8e12f err="Your account may not run more than 2 tunnels over a single ngrok agent session.\nThe tunnels already running on this session are:\ntn_2Kxjngzdu5uiYymfczaXI6nQA6H, tn_2KxjnnYtqzMtZB0msXPr5cMJT7t\n\r\n\r\nERR_NGROK_324\r\n"


PyngrokNgrokHTTPError: ngrok client exception, API returned 502: {"error_code":103,"status_code":502,"msg":"failed to start tunnel","details":{"err":"Your account may not run more than 2 tunnels over a single ngrok agent session.\nThe tunnels already running on this session are:\ntn_2Kxjngzdu5uiYymfczaXI6nQA6H, tn_2KxjnnYtqzMtZB0msXPr5cMJT7t\n\r\n\r\nERR_NGROK_324\r\n"}}
